<a href="https://colab.research.google.com/github/BillWorstell/ISBI2020_TUTORIAL/blob/master/07_segmentation_and_shape_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center">Focused Ion Beam Scanning Electron Microscopy Image Segmentation</h1>


**Summary:**
1. SimpleITK supports a large number of filters that facilitate classical segmentation algorithms (variety of thresholding algorithms, watersheds...).
2. Once your data is segmented SimpleITK enables you to efficiently post process the segmentation (e.g. label distinct objects, analyze object shapes).

This notebook will illustrate the use of SimpleITK for segmentation of bacteria from a 3D Focused Ion Beam Scanning Electron Microscopy (FIB-SEM) image. The specific bacterium is <a href="https://en.wikipedia.org/wiki/Bacillus_subtilis">bacillus subtilis</a>, a rod shaped organism naturally found in soil and plants. The bacteria have been subjected to stress to initiate the process of forming an endospore. These endospores can be seen as a generally dark ellipsoid inside the individual bacterium.

## Install SimpleITK and Environment

Expects to find on MyDrive:
 
/gdrive/MyDrive/SimpleITK-Notebooks/Python

/gdrive/MyDrive/SimpleITK-Notebooks/Python/requirements.txt

/gdrive/MyDrive/Fiji.app/ImageJ-linux64

<img src="figures/ImageOriginAndSpacing.png" style="width:700px"/><br><br>

In [1]:
pip install virtualenv

     |████████████████████████████████| 7.2MB 7.9MB/s 
     |████████████████████████████████| 337kB 31.6MB/s 


In [2]:
#!virtualenv ~/sitkpy --no-site-packages
!virtualenv ~/sitkpy

created virtual environment CPython3.7.10.final.0-64 in 1103ms
  creator CPython3Posix(dest=/root/sitkpy, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==21.0.1, setuptools==52.0.0, wheel==0.36.2
  activators BashActivator,CShellActivator,FishActivator,PowerShellActivator,PythonActivator,XonshActivator


In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [4]:
#!pip install -r '/gdrive/MyDrive/SimpleITK-Notebooks/Python/requirements.txt'
!~/sitkpy/bin/pip install -r '/gdrive/MyDrive/SimpleITK-Notebooks/Python/requirements.txt'

     |████████████████████████████████| 47.4 MB 48 kB/s 
     |████████████████████████████████| 11.5 MB 42.7 MB/s 
     |████████████████████████████████| 121 kB 48.4 MB/s 
     |████████████████████████████████| 15.3 MB 101 kB/s 
     |████████████████████████████████| 27.4 MB 73 kB/s 
     |████████████████████████████████| 9.9 MB 46.4 MB/s 
     |████████████████████████████████| 3.2 MB 37.7 MB/s 
     |████████████████████████████████| 108 kB 70.3 MB/s 
     |████████████████████████████████| 120 kB 78.3 MB/s 
     |████████████████████████████████| 784 kB 64.7 MB/s 
     |████████████████████████████████| 113 kB 57.8 MB/s 
     |████████████████████████████████| 243 kB 58.5 MB/s 
     |████████████████████████████████| 100 kB 6.6 MB/s 
     |████████████████████████████████| 2.2 MB 38.4 MB/s 
     |████████████████████████████████| 428 kB 38.6 MB/s 
     |████████████████████████████████| 108 kB 43.6 MB/s 
     |████████████████████████████████| 59 kB 5.1 MB/s 
     |████████████

In [5]:
cd '/gdrive/MyDrive/SimpleITK-Notebooks/Python'

/gdrive/MyDrive/SimpleITK-Notebooks/Python


In [6]:
pip install SimpleITK

  Using cached https://files.pythonhosted.org/packages/9c/6b/85df5eb3a8059b23a53a9f224476e75473f9bcc0a8583ed1a9c34619f372/SimpleITK-2.0.2-cp37-cp37m-manylinux2010_x86_64.whl


In [7]:
pip install ipywidgets

In [8]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [9]:
%ls -ltr '/gdrive/MyDrive/Fiji.app/ImageJ-linux64'

-rw------- 1 root root 90243 Feb 27 18:15 /gdrive/MyDrive/Fiji.app/ImageJ-linux64


In [10]:
!pip install itk

#!~/sitkpy/bin/pip install ITK

     |████████████████████████████████| 14.4MB 250kB/s 
     |████████████████████████████████| 14.0MB 145kB/s 
     |████████████████████████████████| 50.1MB 67kB/s 
     |████████████████████████████████| 36.3MB 132kB/s 
     |████████████████████████████████| 68.0MB 104kB/s 
     |████████████████████████████████| 10.3MB 20.1MB/s 


In [11]:
!echo $PATH

/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin


In [12]:
import os
#os.environ['PYTHONPATH'] += ":/content/gdrive/My Drive/Colab Notebooks/MNIST_Classifier/src"
os.environ['PATH'] += ":/content/gdrive/My Drive/Fiji.app"
! echo $PATH

/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin:/content/gdrive/My Drive/Fiji.app


In [13]:
cd '/gdrive/MyDrive/SimpleITK-Notebooks/Python'

/gdrive/MyDrive/SimpleITK-Notebooks/Python


In [14]:
import SimpleITK as sitk

%run update_path_to_download_script
from downloaddata import fetch_data, fetch_data_all

from ipywidgets import interact

print(sitk.Version())

SimpleITK Version: 2.0.2 (ITK 5.1)
Compiled: Dec  1 2020 22:01:03



In [15]:
cd /usr/local/

/usr/local


In [16]:
import shutil

In [17]:
%ls /gdrive/MyDrive/

'Colab Notebooks'/                      SimpleITK_ISBI2020_TUTORIAL/
 Fiji.app/                              SimpleITK.jpg
 fiji-linux64-20170530.zip              SimpleITK-Notebooks/
 MyNiftyRec/                            unnamed.png
 MyTomoLab/                            'Untitled project.gscript'
 pyjnius-1.2.0-py37h90b5fae_0.tar.bz2


In [18]:
shutil.copyfile('/gdrive/MyDrive/fiji-linux64-20170530.zip', '/usr/local/fiji-linux64-20170530.zip')

'/usr/local/fiji-linux64-20170530.zip'

In [19]:
cd /usr/local

/usr/local


In [20]:
!unzip ./fiji-linux64-20170530.zip

Archive:  ./fiji-linux64-20170530.zip
  inflating: Fiji.app/db.xml.gz      
  inflating: Fiji.app/Contents/Info.plist  
  inflating: Fiji.app/Contents/Resources/Fiji.icns  
  inflating: Fiji.app/Contents/Resources/ImageJ.icns  
  inflating: Fiji.app/ImageJ-linux64  
  inflating: Fiji.app/plugins/3D_Blob_Segmentation-3.0.0.jar  
  inflating: Fiji.app/plugins/3D_Objects_Counter-2.0.0.jar  
  inflating: Fiji.app/plugins/3D_Viewer-4.0.1.jar  
  inflating: Fiji.app/plugins/Algorithm_Launcher.jar  
  inflating: Fiji.app/plugins/Analyze/Dynamic_ROI_Profiler.clj  
  inflating: Fiji.app/plugins/Analyze/Measure_RGB.txt  
  inflating: Fiji.app/plugins/AnalyzeSkeleton_-3.1.2.jar  
  inflating: Fiji.app/plugins/Anisotropic_Diffusion_2D-2.0.0.jar  
  inflating: Fiji.app/plugins/Archipelago_Plugins-0.5.2.jar  
  inflating: Fiji.app/plugins/Arrow_-2.0.1.jar  
  inflating: Fiji.app/plugins/Auto_Threshold-1.16.4.jar  
  inflating: Fiji.app/plugins/BalloonSegmentation_-3.0.0.jar  
  inflating: Fiji.app/p

In [21]:
#!pip install SimpleITK

In [22]:
#!git clone https://github.com/InsightSoftwareConsortium/SimpleITK-Notebooks.git

In [23]:
pip install ipywidgets

In [24]:
# this will allow the notebook to reload/refresh automatically within the runtime
%reload_ext autoreload
%autoreload 2

from ipywidgets import interact

def f(x):
  return x

interact(f, x=10)

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.f>

In [25]:
#pip install itk

In [26]:
cd '/gdrive/MyDrive/SimpleITK-Notebooks/Python/'

/gdrive/MyDrive/SimpleITK-Notebooks/Python


In [27]:
!python '/gdrive/MyDrive/SimpleITK-Notebooks/Python/update_path_to_download_script.py'

In [28]:
import os
os.environ['PYTHONPATH'] += ":/content/gdrive/My Drive/Fiji.app"
! echo $PYTHONPATH
os.environ['PATH'] += ":/content/gdrive/My Drive/Fiji.app"
! echo $PATH

/env/python:/content/gdrive/My Drive/Fiji.app
/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin:/content/gdrive/My Drive/Fiji.app:/content/gdrive/My Drive/Fiji.app


In [29]:
import numpy as np
import os
from ipywidgets import interact, fixed

%matplotlib inline
import matplotlib.pyplot as plt

from downloaddata import fetch_data as fdata

OUTPUT_DIR = 'output'

image_viewer = sitk.ImageViewer()

In [30]:
print(image_viewer)

itk::simple::ImageViewer
  Title: 
  Command: %a -eval 'open("%f"); rename("%t");'
  Application: /usr/local/Fiji.app/ImageJ-linux64
  Default Application: /usr/local/Fiji.app/ImageJ-linux64
  File Extension: .mha
  Default File Extension: .mha
  Search Path: [ ./, /root/bin/, /opt/, /usr/local/ ]
  Executable Names: [ Fiji.app/ImageJ-linux64, Fiji.app/ImageJ-linux32 ]
  Debug Flag: 0



In [31]:
%ls /usr/local/Fiji.app/ImageJ-linux64

/usr/local/Fiji.app/ImageJ-linux64*


In [32]:
#!python '/gdrive/MyDrive/SimpleITK-Notebooks/Python/update_path_to_download_script.py'

In [33]:
cd '/gdrive/MyDrive/SimpleITK-Notebooks/Python/'

/gdrive/MyDrive/SimpleITK-Notebooks/Python


In [34]:
import SimpleITK as sitk

%run update_path_to_download_script
from downloaddata import fetch_data, fetch_data_all

from ipywidgets import interact

print(sitk.Version())

SimpleITK Version: 2.0.2 (ITK 5.1)
Compiled: Dec  1 2020 22:01:03



In [35]:
%cd '/gdrive/MyDrive/'
%ls

/gdrive/MyDrive
'Colab Notebooks'/                      SimpleITK_ISBI2020_TUTORIAL/
 Fiji.app/                              SimpleITK.jpg
 fiji-linux64-20170530.zip              SimpleITK-Notebooks/
 MyNiftyRec/                            unnamed.png
 MyTomoLab/                            'Untitled project.gscript'
 pyjnius-1.2.0-py37h90b5fae_0.tar.bz2


In [36]:
cd /usr/local/

/usr/local


In [37]:
#image_viewer.SetApplication('/usr/local/Fiji.app/ImageJ-linux64')

In [38]:
# Uncomment the line below to change the default external viewer to your viewer of choice and test that it works.
#%env SITK_SHOW_COMMAND /Applications/ITK-SNAP.app/Contents/MacOS/ITK-SNAP 
#%env SITK_SHOW_COMMAND  /gdrive/MyDrive/Fiji.app/ImageJ-linux64
#%env SITK_SHOW_COMMAND  /usr/local/Fiji.app/
#%env SITK_SHOW_COMMAND  /usr/local/Fiji.app/ImageJ-linux64

# Retrieve an image from the network, read it and display using the external viewer. 
# The show method will also set the display window's title and by setting debugOn to True, 
# will also print information with respect to the command it is attempting to invoke.
# NOTE: The debug information is printed to the terminal from which you launched the notebook
#       server.

In [39]:
%ls /gdrive/MyDrive/

'Colab Notebooks'/                      SimpleITK_ISBI2020_TUTORIAL/
 Fiji.app/                              SimpleITK.jpg
 fiji-linux64-20170530.zip              SimpleITK-Notebooks/
 MyNiftyRec/                            unnamed.png
 MyTomoLab/                            'Untitled project.gscript'
 pyjnius-1.2.0-py37h90b5fae_0.tar.bz2


In [40]:
%cd /gdrive/MyDrive/SimpleITK_ISBI2020_TUTORIAL

/gdrive/MyDrive/SimpleITK_ISBI2020_TUTORIAL


In [41]:
print(image_viewer)

itk::simple::ImageViewer
  Title: 
  Command: %a -eval 'open("%f"); rename("%t");'
  Application: /usr/local/Fiji.app/ImageJ-linux64
  Default Application: /usr/local/Fiji.app/ImageJ-linux64
  File Extension: .mha
  Default File Extension: .mha
  Search Path: [ ./, /root/bin/, /opt/, /usr/local/ ]
  Executable Names: [ Fiji.app/ImageJ-linux64, Fiji.app/ImageJ-linux32 ]
  Debug Flag: 0



In [42]:
#import SimpleITK as sitk

#%matplotlib inline
#import matplotlib.pyplot as plt
#import numpy as np

#from ipywidgets import interact, fixed
#import os

#OUTPUT_DIR = 'Output'

# Utility method that either downloads data from the Girder repository or
# if already downloaded returns the file name for reading from disk (cached data).
#%run update_path_to_download_script
#from downloaddata import fetch_data as fdata

In [43]:
import SimpleITK as sitk
import pandas as pd

%matplotlib notebook

import matplotlib.pyplot as plt
import gui
from math import ceil
from downloaddata import fetch_data as fdata

# Load data

Load the 3D volume and display it.

In [44]:
img = sitk.ReadImage(fdata("fib_sem_bacillus_subtilis.mha"))
gui.MultiImageDisplay(image_list = [img], figure_size=(8,4));

Fetching fib_sem_bacillus_subtilis.mha
Downloaded 74215399 of 74215399 bytes (100.00%)


<IPython.core.display.Javascript object>

# Segmentation

To allow us to analyze the shape of whole bacteria we first need to segment them. We will do this in several steps:
1. Separate the bacteria from the embedding resin background.
2. Mark each potential bacterium with a unique label, to evaluate the segmentation.
3. Remove small components and fill small holes using binary morphology operators (opening and closing).
4. Use seed based watersheds to perform final segmentation.
5. Remove bacterium that are connected to the image boundary.

## Separate the bacteria from the background

Based on the visualization of the data above, it intuitively appears that the background and foreground are separable using a single intensity threshold. Our first step towards validating this observation is to plot the intensity distribution.

In [45]:
plt.figure()
plt.hist(sitk.GetArrayViewFromImage(img).flatten(), bins=100)
plt.show()

<IPython.core.display.Javascript object>

The histogram is bi-modal with a clear separation, which we have manually identified as having an intensity value of 120.

We can also use one of several binary threshold selection filters available in SimpleITK. 

In [46]:
threshold_filters = {'Otsu': sitk.OtsuThresholdImageFilter(),
                     'Triangle' : sitk.TriangleThresholdImageFilter(),
                     'Huang' : sitk.HuangThresholdImageFilter(),
                     'MaxEntropy' : sitk.MaximumEntropyThresholdImageFilter()}

filter_selection = 'Manual'
try:
  thresh_filter = threshold_filters[filter_selection]
  thresh_filter.SetInsideValue(0)
  thresh_filter.SetOutsideValue(1)
  thresh_img = thresh_filter.Execute(img)
  thresh_value = thresh_filter.GetThreshold()
except KeyError:
  thresh_value = 120
  thresh_img = img>thresh_value

print("Threshold used: " + str(thresh_value))    
gui.MultiImageDisplay(image_list = [sitk.LabelOverlay(img, thresh_img)],                   
                      title_list = ['Binary Segmentation'], figure_size=(8,4));

Threshold used: 120


<IPython.core.display.Javascript object>

# Mark each potential bacterium with unique label and evaluate

In [47]:
stats = sitk.LabelShapeStatisticsImageFilter()
stats.Execute(sitk.ConnectedComponent(thresh_img))

# Look at the distribution of sizes of connected components (bacteria).
label_sizes = [ stats.GetNumberOfPixels(l) for l in stats.GetLabels() if l != 1]

plt.figure()
plt.hist(label_sizes,bins=200)
plt.title("Distribution of Object Sizes")
plt.xlabel("size in pixels")
plt.ylabel("number of objects")
plt.show()

<IPython.core.display.Javascript object>

The histogram above shows tens of thousands of very small labels which are not visually detected by looking at the segmentation.

## Remove small islands and holes

Using binary morphological operations we remove small objects using the opening operation and fill small holes using the closing operation. The use of opening and closing by reconstruction maintains the boundary of the original objects.

In [ ]:
cleaned_thresh_img = sitk.BinaryOpeningByReconstruction(thresh_img, [10, 10, 10])
cleaned_thresh_img = sitk.BinaryClosingByReconstruction(cleaned_thresh_img, [10, 10, 10])

gui.MultiImageDisplay(image_list = [sitk.LabelOverlay(img, cleaned_thresh_img)],                   
                      title_list = ['Cleaned Binary Segmentation'], figure_size=(8,4));

Check that the number of objects defined by the binary image is more reasonable.

In [ ]:
stats = sitk.LabelShapeStatisticsImageFilter()
stats.Execute(sitk.ConnectedComponent(cleaned_thresh_img))

# Look at the distribution of sizes of connected components (bacteria).
label_sizes = [ stats.GetNumberOfPixels(l) for l in stats.GetLabels() if l != 1]

plt.figure()
plt.hist(label_sizes,bins=200)
plt.title("Distribution of Object Sizes")
plt.xlabel("size in pixels")
plt.ylabel("number of objects")
plt.show()

After the morphological operations, our binary image seems to have a reasonable number of objects, but is this true? We next look at the unique objects defined by this binary segmentation (each object is marked with a unique color).

In [ ]:
gui.MultiImageDisplay(image_list = [sitk.LabelOverlay(img, sitk.ConnectedComponent(cleaned_thresh_img))],                   
                      title_list = ['Cleaned Binary Segmentation'],figure_size=(8,4));

## Seed based watershed segmentation

The bacteria appear to be segmented correctly from the background but not from each other. Using the visualization and histogram above we see that in 3D many of them are connected, even if on a slice by slice inspection they appear separate.  

In [ ]:
dist_img = sitk.SignedMaurerDistanceMap(cleaned_thresh_img != 0, insideIsPositive=False, squaredDistance=False, useImageSpacing=False)
radius = 10
# Seeds have a distance of "radius" or more to the object boundary, they are uniquely labelled.
seeds = sitk.ConnectedComponent(dist_img < -radius)
# Relabel the seed objects using consecutive object labels while removing all objects with less than 15 pixels.
seeds = sitk.RelabelComponent(seeds, minimumObjectSize=15)
# Run the watershed segmentation using the distance map and seeds.
ws = sitk.MorphologicalWatershedFromMarkers(dist_img, seeds, markWatershedLine=True)
ws = sitk.Mask( ws, sitk.Cast(cleaned_thresh_img, ws.GetPixelID()))

Visualize the distance map, the unique seeds and final object segmentation.

In [ ]:
gui.MultiImageDisplay(image_list = [dist_img,
                                    sitk.LabelOverlay(img, seeds),
                                    sitk.LabelOverlay(img, ws)],                   
                      title_list = ['Segmentation Distance',
                                    'Watershed Seeds',
                                    'Binary Watershed Labeling'],
                      shared_slider=True,
                      horizontal=False,
                      figure_size=(6,12));

## Removal of objects touching the image boundary

We are not sure objects touching the image boundary are whole bacteria, so we remove them.

In [ ]:
# The image has a small black border which we account for here.
bgp = sitk.BinaryGrindPeak( (ws!=0)| (img==0))
non_border_seg = sitk.Mask( ws, bgp==0)
gui.MultiImageDisplay(image_list = [sitk.LabelOverlay(img, non_border_seg)],                   
                      title_list = ['Final Segmentation'],figure_size=(8,4));

# Object Analysis

Once we have the segmented objects we look at their shapes and the intensity distributions inside the objects.

Note that sizes are in nanometers. ITK and consequently SimpleITK are agnostic of the actual measurement units. It is up to you as the developer to explicitly use the correct units and more importantly, <a href="https://en.wikipedia.org/wiki/Mars_Climate_Orbiter">DO NOT MIX UNITS</a>.

We first compute all of the measurements we are interested in.

In [ ]:
shape_stats = sitk.LabelShapeStatisticsImageFilter()
shape_stats.ComputeOrientedBoundingBoxOn()
shape_stats.Execute(non_border_seg)

intensity_stats = sitk.LabelIntensityStatisticsImageFilter()
intensity_stats.Execute(non_border_seg,img) 

Insert the values into a pandas dataframe and display some descriptive statistics.

In [ ]:
stats_list = [ (shape_stats.GetPhysicalSize(i),
               shape_stats.GetElongation(i),
               shape_stats.GetFlatness(i),
               shape_stats.GetOrientedBoundingBoxSize(i)[0],
               shape_stats.GetOrientedBoundingBoxSize(i)[2],
               intensity_stats.GetMean(i),
               intensity_stats.GetStandardDeviation(i),
               intensity_stats.GetSkewness(i)) for i in shape_stats.GetLabels()]
cols=["Volume (nm^3)",
      "Elongation",
      "Flatness",
      "Oriented Bounding Box Minimum Size(nm)",
      "Oriented Bounding Box Maximum Size(nm)",
     "Intensity Mean",
     "Intensity Standard Deviation",
     "Intensity Skewness"]

# Create the pandas data frame and display descriptive statistics.
stats = pd.DataFrame(data=stats_list, index=shape_stats.GetLabels(), columns=cols)
stats.describe()

Create a plot to investigate the relationship, possible correlations, between volume and object shape characteristics (elongation, flatness, principal moments). 

In [ ]:
fig, axes = plt.subplots(nrows=len(cols), ncols=2, figsize=(6,4*len(cols)))
axes[0,0].axis('off')

stats.loc[:,cols[0]].plot.hist(ax=axes[0,1], bins=25)
axes[0,1].set_xlabel(cols[0])
axes[0,1].xaxis.set_label_position("top")

for i in range(1,len(cols)):
    c = cols[i]
    bar = stats.loc[:,[c]].plot.hist(ax=axes[i,0], bins=20,orientation='horizontal',legend=False)
    bar.set_ylabel(stats.loc[:,[c]].columns.values[0])    
    scatter = stats.plot.scatter(ax=axes[i,1],y=c,x=cols[0])
    scatter.set_ylabel('')
    # Remove axis labels from all plots except the last (they all share the labels)
    if(i<len(cols)-1):
        bar.set_xlabel('')
        scatter.set_xlabel('')
# Adjust the spacing between plot columns and set the plots to have a tight
# layout inside the figure.
plt.subplots_adjust(wspace=0.4)
plt.tight_layout()

Finally, we visualize a lineup of the bacteria using a coordinate system that is defined by the oriented bounding box enclosing each of them. 

In [ ]:
bacteria_labels = shape_stats.GetLabels()
bacteria_volumes = [shape_stats.GetPhysicalSize(label) for label in bacteria_labels] 
num_images = 5 # number of bacteria images we want to display

bacteria_labels_volume_sorted = [label for _,label in sorted(zip(bacteria_volumes, bacteria_labels))]

resampler = sitk.ResampleImageFilter()
aligned_image_spacing = [10,10,10] #in nanometers

for label in bacteria_labels_volume_sorted[0:num_images]:
    aligned_image_size = [ int(ceil(shape_stats.GetOrientedBoundingBoxSize(label)[i]/aligned_image_spacing[i])) for i in range(3) ]
    direction_mat = shape_stats.GetOrientedBoundingBoxDirection(label)
    aligned_image_direction = [direction_mat[0], direction_mat[3], direction_mat[6], 
                               direction_mat[1], direction_mat[4], direction_mat[7],
                               direction_mat[2], direction_mat[5], direction_mat[8] ] 
    resampler.SetOutputDirection(aligned_image_direction)
    resampler.SetOutputOrigin(shape_stats.GetOrientedBoundingBoxOrigin(label))
    resampler.SetOutputSpacing(aligned_image_spacing)
    resampler.SetSize(aligned_image_size)
    
    obb_img = resampler.Execute(img)
    # Change the image axes order so that we have a nice display.
    obb_img = sitk.PermuteAxes(obb_img,[2,1,0])
    gui.MultiImageDisplay(image_list = [obb_img],                   
                          title_list = ["OBB_{0}".format(label)])

<a href="08_segmentation_evaluation.ipynb"><h2 align=right>Next &raquo;</h2></a>